#Loading RAW data from bronze Layer (ERP)

## Init


In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import StringType

##Importing loc_a101 Data

In [0]:
df=spark.table("workspace.bronze.erp_loc_a101")
df.display()

###Cleaning up loc_a101 data

In [0]:
for field in df.schema.fields:
    if isinstance(field.dataType,StringType):
        df=df.withColumn(field.name,trim(col(field.name)))
# df.display()

###Data Processing and Renaming Columns

In [0]:
df=(
    df.withColumn("CNTRY",expr("""
                                CASE 
                                WHEN CNTRY = '' OR CNTRY IS NULL THEN 'n/a'
                                WHEN UPPER(CNTRY) = 'USA' OR UPPER(CNTRY)='US' THEN 'United States'
                                WHEN UPPER(CNTRY) = 'CAN' THEN 'Canada'
                                WHEN UPPER(CNTRY) = 'DE' THEN 'Germany'
                                WHEN UPPER(CNTRY) = 'FR' THEN 'France'
                                ELSE CNTRY
                                END
                               """))
    .withColumn("CID",expr("REPLACE(CID,'-','')"))
    .withColumnRenamed("CNTRY","country")
    .withColumnRenamed("CID","customer_id")
)
df.display()

### Loading erp_loc_a101 Data  

In [0]:
(
  df.write
  .mode("overwrite")
  .format("delta")
  .saveAsTable("silver.erp_loc_az12")
)